## Basic relationship plots

Last time, we played around with plotting the distributions of variables, and comparing distributions to one another. Sometimes, however two variables are more intimately related such that knowing a particular value of one variable allows you to predict, to some extent, the value of another variable. Simply plotting the values of one variable against the values of the other is best way to begin to appreciate the nature and strength of the relationship. So let's play around with plotting and evaluation some relationships.

##### As always, we'll start by importing the libraries we'll use.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

##### And read the first of our data files

In [ ]:
mdff = pd.read_csv("datasets/008TutorialDataFile1.csv")

##### Now let's take a look to make sure we read something that looks okay

In [ ]:
display(mdff)

#### We can also check this visually using the data frame's built in plotting capability.

In [ ]:
mdff.plot(x='X', y='Y', kind='scatter', color='r')

That looks a straight line relationship in which y is related to x by `y = a + bx + err`, in which *a* is y-intercept, *b* is the slope, and *err* represents 'error' or 'noise', that is, variablility in y that is unrelated to x. 
So, in a nutshell, it looks like if we know the value of x, we can predict the value of why except for a some amount of random variable represented by *err*. And all we need to know to do the predicting is two numbers, a slope and a y intercept.

Now lets do a prettier plot using seaborn.

In [ ]:
sns.set_theme() # set theme to seaborn's default

Just like `displot()` is the seaborn quick and easy way for plotting distributions of variables, `relplot()` is the Q&E way for plotting relationships among variables. 

In [ ]:
sns.relplot(data=mdff, x='X', y='Y')

As we noted above, the data seem to fall around a roughly straight line. We can easily fit and plot that line using the `seaborn.lmplot()` function, where the "lm" stands for "linear model" (of which a straight line is the simplest and default case).

In [ ]:
sns.lmplot(data=mdff, x='X', y='Y')

In this plot, there is a straight line plotted along with the data with the particular slope and y-intercept that do the best job of simultaneously coming the closest to all the data. More specifically, it is the line than minimizes the *sum of the squared differences between the line and the y data values*, called the *sum squared error* or just the *sse*. Also plotted is an error bound around fit. It means that, assuming the samples we have are representative of the underlying populations, or whatever the underlying phenomenon are that are producing the data, then 95% of experiments would produce a straight line fit somewhere within these bounds. 

There are two main things we can take away from this plot. First, there is a relationship between the variables, and the data nail down that relationship very well (as indicated by the small error bounds).

Second, there is noise – random variability – that limit our ability to predict *any one particular* y value from a give x value. 

We can estimate the slope and y intercept of the straight line relationship with `numpy.polyfit()' as follows:

In [ ]:
myfit = np.polyfit(x=mdff['X'], y=mdff['Y'], deg=1)
print(myfit)

The `deg=1` argument specifies that we want to fit the data with a *first order polynomial*, i.e., a straight line. The returned coefficients tell us that the slope is around 4.5 and the y intercept is around 240. But, of course, these numbers only allow is to predict y *on average*. There is also the error, which puts a fundamental limit on our ability to predict any particular y value.

We can explore the error a little more by looking at the *residuals*, which is just the difference (literally) between the line and the y values. 

Seaborn provides an easy way to peek at the rediduals using `seaborn.residplot()`.

In [ ]:
sns.residplot(data=mdff, x='X', y='Y')

We can think of this as a picture of the noise alone; we have literally subtracted out the linear relationship with x! The range of the noise looks to be around 200 total in the y direction. If the variability is Gaussian, this would correspond to a standard deviation of around 200/6 or 33 (do you see why we divided by 6?). So we can say that we can predict y *on average* with a precision corresponding to a sigma of about +/- 33.

Let's explore this a little further by looking at the actual distribution of the residuals. In other words, lets look at the distribution of the noise component of y. We can do this by first getting the actual values of our best fit line using `np.polyval()` – we just give it our fit from `np.polyfit()` and our x values.

In [ ]:
fitvals = np.polyval(myfit, mdff['X'])

Now that we have those, we can compute the residual values by subtracting the best fit line from the y values.

In [ ]:
myres = mdff['Y'] - fitvals

And then plotting those!

In [ ]:
sns.displot(myres, kind='kde')

Sure enough, looks like a Gaussian distribution with a standard deviation of around 30 or so. Or, more precisely:

In [ ]:
np.std(myres)

Okay, so, what have we done? We have looked at the data and then created a simple model in which y is a linear function of x plus random variability. If we let *N*(mu, sigma) stand for a normal distribution with a mean of mu and a standard deviation of sigma, the we can actually write down our as

 y = 4.5 * x + 242 + *N*(0, 36)

Even though this is a simple line fit, the basic procedure we just followed is the same no matter how complicated a situation we are dealing with:

* look at the data
* make a guess at the relationship (perhaps using prior knowledge in addition to the data)
* fit a model to the data
* evaluate how well the model fits the data

Sometimes these steps are informal and internal (Oh, yeah, that's linear), and sometimes we go way down into the weeds in the fitting and evaluation, but these are the basic steps!

##### Let's look at a second set of data!

In [ ]:
mdff = pd.read_csv("datasets/008TutorialDataFile2.csv")

Take a peek:

In [ ]:
display(mdff)

Plot the relationship!

In [ ]:
sns.relplot(data=mdff, x='X', y='Y')

So these data look both the same and different than the last data set. They look the same in that it looks like there is a linear relationship between x and y. Let's look at that.

In [ ]:
sns.lmplot(data=mdff, x='X', y='Y')

So it looks like we have a nice linear relationship as before, but the slope is perhaps not pinned down as well (note the bow tie shape of the error bounds). Cool.

But these data also look different if we look at the x values. In the last data set, the x values were evenly spaced, as though they came from a laboratory experiment in which x was intentionally adjusted in a precise way. In this data set, it looks as though x, like y, was randomly sampled.

So let's plot these data in some ways that we can look at both x and y by making a scatterplot and then adding a "rug" plot along the axes.

In [ ]:
sns.scatterplot(data=mdff, x='X', y='Y')
sns.rugplot(data=mdff, x='X', y='Y')

The rugs are basically interior tic marks showing the positions of each data point. Here, we can see that both variables cluster near the centers and get more sparse towards the edges of the plot.

Because both x and y seem to be random variables, the scatter plot above shows the *joint distribution* of x and y. We can take a more detailed look by plotting both the joint distribution, and the individual or *"marginal"* distributions of the two variables in the *margins* of the plot. 

In [ ]:
sns.jointplot(x="X", y="Y", data=mdff)

Oh! That's pretty! And we can see at a glance that each variable is distributed roughly as Gaussians as well as seeing the y vs. x relationship. We can also make a version with the best fit line by specifying the `kind` argument to `reg` for regression.

In [ ]:
sns.jointplot(x="X", y="Y", data=mdff, kind="reg")

As a bonus, this also seems to add KDE plots to the marginal distributions!

So, looking at these data, we'd conclude that there is a positive linear relationship between x and y, though both variables are noisy. If we wished, we could of course do a deeper dive into a linear model and how well it fits!

##### Let's look at our third and final dataset.

In [ ]:
mdff = pd.read_csv("datasets/008TutorialDataFile3.csv")
display(mdff)

In [ ]:
sns.relplot(data=mdff, x='X', y='Y')

So, again, we have a strong relationship. Let's do our plot with a simple model:

In [ ]:
sns.lmplot(data=mdff, x='X', y='Y')

The simple line fit looks okay, and the error bounds on the line are small. Great! Right!

If we look a little more closely at the actual data though, without getting seduced by the line, we see that the relationship here is perhaps a little more complicated. In addition to y going up with x, it looks like there's a curve in the data such that, the higher x is, the faster y increases.

Let's look at the residuals from our line fit.

In [ ]:
sns.residplot(data=mdff, x='X', y='Y')

Here we can clearly see that there is a *pattern to the residuals*. This is diagnostic. In general, true error, true random variability tends to be normally distributed (thank you Central Limit Theorem). Thus, if our model is really capturing the data, then the residuals should be flat and normally distributed around zero (think about it). So this "smile" pattern in the data is because our model is systematically overestimating the data in the middle and underestimating at the two extremes.

It looks like the data are bending but our model isn't.

We'll talk about different kinds of models as we go on, but a very simple way to capture a bend in data is to expand our *first degree polynomial* model – a straight line – to a *second order polynomial* in which

 y = a + bx + cx^2 + *N*(0,1)

 The squared term turns our carrot model into a banana model. Let's look!

In [ ]:
sns.lmplot(data=mdff, x='X', y='Y', order=2)

Note the `order=2` argument in `lmplot()`

You might be thinking "Wait, didn't "lm" stand for "linear model? But now we're squaring x, doesn't that make it non-linear"? Indeed, all polynomials are actually linear models by definition – not all lines are straight!

Let's look at the residuals again!

In [ ]:
g = sns.residplot(data=mdff, x='X', y='Y', order=2)

Okay, that looks good now! But let's go ahead and take a slightly deeper dive like we did above, but without the color commentary.

In [ ]:
myfit = np.polyfit(x=mdff['X'], y=mdff['Y'], deg=2)
print(myfit)

In [ ]:
fitvals = np.polyval(myfit, mdff['X'])

In [ ]:
myres = mdff['Y'] - fitvals

In [ ]:
sns.displot(myres)

In [ ]:
np.std(myres)

So now we can say that the residuals no look like truly random noise, and that the data are captured by

 y = 7.3 + 0.02x + 0.7x^2 + *N*(0,98)
 

So, here, we played all the same games that we talked about above. The difference is that we tried a candidate model, a straight line, decided it wasn't quite right, and then settled on a curvy model instead.

Now, as might have realized, the models we played with today were purely *descriptive*. We don't know what "x" and "y" are, and can't therefor say anything about *why* y should be related to x. Ultimately, we want models that represent the things that are generating the data, not just ones that reasonably describe the data.

Ultimately, though, the basic process of fitting and evaluation are the same, so this tutorial gives us a good start!